<small><font color=gray>Notebook author: <a href="https://www.linkedin.com/in/olegmelnikov/" target="_blank">Oleg Melnikov</a>

**[<font size=6>💳Mr. and Mrs. Client (MaMC)</font>](https://www.kaggle.com/competitions/26jun23/rules)**. [**Instructions**](https://colab.research.google.com/drive/1riOGrE_Fv-yfIbM5V4pgJx4DWcd92cZr#scrollTo=ITaPDPIQEgXV) for running Colabs.

<small>**(Optional) CONSENT.** <mark>[  ]</mark> We consent to sharing our Colab (after the assignment ends) with other students/instructors for educational purposes. We understand that sharing is optional and this decision will not affect our grade in any way. <font color=gray><i>(If ok with sharing your Colab for educational purposes, leave "X" in the check box.)</i></font></small>

In [ ]:
from google.colab import drive; drive.mount('/content/drive')   # OK to enable, if your kaggle.json is stored in Google Drive

Mounted at /content/drive


In [ ]:
!pip -q install --upgrade --force-reinstall --no-deps kaggle > log  # upgrade kaggle package (to avoid a warning)
!mkdir -p ~/.kaggle                                           # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >log  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
!cp kaggle.json ~/.kaggle/kaggle.json > log                   # Alternative location of kaggle.json (without a connection to Google Drive)
!chmod 600 ~/.kaggle/kaggle.json                              # give only the owner full read/write access to kaggle.json
!kaggle config set -n competition -v 26jun23                  # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
!kaggle competitions download >> log                          # download competition dataset as a zip file
!unzip -o *.zip >> log                                        # Kaggle dataset is copied as a single file and needs to be unzipped.
!kaggle competitions leaderboard --show                       # print public leaderboard

cp: cannot stat 'kaggle.json': No such file or directory
- competition is now set to: 26jun23
100% 35.7M/35.7M [00:00<00:00, 308MB/s]
Using competition: 26jun23
  teamId  teamName                       submissionDate       score    
--------  -----------------------------  -------------------  -------  
10612706  Michael Kondrashov             2023-06-30 11:34:09  0.88424  
10612695  Logistic Depression [12]       2023-06-30 12:39:10  0.88402  
10614699  Dmitriy Geyzha                 2023-06-30 11:59:52  0.87769  
10616508  Zа БVИ [8]                     2023-06-30 12:24:49  0.87710  
10614521  Konstantin Zeigman             2023-06-30 11:37:17  0.87691  
10612698  Бобы                           2023-06-30 10:43:59  0.87539  
10617156  Team 5                         2023-06-30 11:08:02  0.87355  
10618717  Team1                          2023-06-30 12:29:16  0.87097  
10613713  Mikhail Vovk                   2023-06-30 12:35:15  0.86481  
10612703  Пельмени (Хомяков и Ткачев)    2023-0

In [ ]:
%%time
%%capture
%reset -f
!pip install -U sentence-transformers tfds-nightly tensorflow tensorflow_addons plotly > log
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all"  # allows multiple prints from a cell
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, os, plotly.express as px, xgboost as xgb, lightgbm, re, tensorflow as tf, tensorflow.keras as keras
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, GroupKFold
from sklearn.linear_model import LogisticRegression as LR
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier   # SKLearn's MLP is optimised for CPU (and doesn't use GPU)
from sentence_transformers import SentenceTransformer as SBERT
from keras.layers import Flatten, Dense

os.environ['TF_DETERMINISTIC_OPS'] = '1'; os.environ['TF_CUDNN_DETERMINISTIC'] = '1'   # helps to seed TF's RNG on GPUs
np.set_printoptions(linewidth=10000, precision=2, edgeitems=20, suppress=True)
pd.set_option('display.max_colwidth', 100, 'display.max_columns', 10, 'display.width', 1000, 'display.max_rows', 8)

class Timer():
  def __init__(self, lim:'RunTimeLimit'=60*2): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

CPU times: user 8.18 s, sys: 1.01 s, total: 9.19 s
Wall time: 1min 12s


In [ ]:
%time mcc = pd.read_csv('_mcc.csv', index_col='mcc'); mcc.T  # load merchant category codes (MCC)

CPU times: user 5.39 ms, sys: 774 µs, total: 6.16 ms
Wall time: 23.8 ms


mcc,0,1,2,3,4,...,179,180,181,182,183
desc,Ветеринарные услуги,"Генеральные подрядчики по вентиляции, теплоснабжению, и водопроводу",Подрядчики по электричеству,"Подрядчики, специализированная торговля — нигде более не классифицированные",Разнообразные издательства/печатное дело,...,"Судовые выплаты, включая алименты и детскую поддержку",Штрафы,Налоговые платежи,"Правительственные услуги, нигде ранее не классифицируемые",Почтовые услуги — только правительство


In [ ]:
%time ttc = pd.read_csv('_ttc.csv', index_col='ttc'); ttc.T  # load transaction type codes (TTC)

CPU times: user 3.98 ms, sys: 0 ns, total: 3.98 ms
Wall time: 4.97 ms


ttc,0,1,2,3,4,...,150,151,152,153,154
desc,Покупка.ТУ СБ РФ,Покупка. POS ТУ СБ РФ,Оплата услуги. Банкоматы СБ РФ,Покупка. ТУ Россия,Покупка. POS ТУ Россия,...,Урегулирование безнадежной задолженности,Плата за приост. действия карты,Плата за обработку счета банковской карты клиента при осущ. платежной системой экстренной выдачи...,Плата за предоставление по запросу клиента документов по операции с картой,XXX


In [ ]:
%time gender = pd.read_csv('gender.csv', index_col='cid'); gender.T  # load gender labels. you need to predict the missing labels
tY, vY = gender.query('gender==gender'), gender.query('gender!=gender')  # split into Train and test (=Validation) targets

CPU times: user 4.22 ms, sys: 20 µs, total: 4.24 ms
Wall time: 4.53 ms


cid,0,1,2,3,4,...,8395,8396,8397,8398,8399
gender,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,0.0,0.0


In [ ]:
%time dfTrx = pd.read_csv('trx.csv').sort_values(['cid','dt','tm']); dfTrx.T  # load bank card's transactions. Ensure ordered CID

CPU times: user 17.2 s, sys: 1.55 s, total: 18.7 s
Wall time: 29.1 s


,0,1,2,3,4,...,3749573,3749574,3749575,3749576,3749577
cid,0,0,0,0,0,...,8399,8399,8399,8399,8399
dt,1,1,2,3,3,...,450,450,456,456,456
tm,00:00:00,00:00:00,22:48:17,00:00:00,00:00:00,...,00:00:00,00:00:00,17:21:07,18:59:26,19:01:21
amt,-1305,-4821,-56156,-3708,-30912,...,-21877,-43831,89833,247062,-336874
mcc,56,56,22,56,65,...,56,56,123,124,22
ttc,4,4,37,4,4,...,4,4,128,108,25
tid,NaN,NaN,NaN,NaN,NaN,...,25021545,PS026577,888917,738051,738051


In [ ]:
df_ = gender.drop('gender', axis=1); df_.T  # a list of ordered client IDs (as index)

cid,0,1,2,3,4,...,8395,8396,8397,8398,8399


In [ ]:
pd.set_option('display.max_colwidth', 100, 'display.max_columns', 20, 'display.width', 1000, 'display.max_rows', 4)
tmr = Timer() # runtime limit (in seconds). Add all of your code after the timer

⏳ started. You have 120 sec. Good luck!


<hr color=green size=40>

<strong><font color=green size=5>⏳Timed Green Playground (TGP): Your ideas, documentation, code, and timer START HERE!</font></strong>

<font color=green>Students: Keep all your definitions, code, documentation in <b>TGP</b>. Modifying any code outside of TGP incurs penalties.

# Formatting and feature engineering

Ignoring all warnings raised by pandas

In [ ]:
import warnings
warnings.filterwarnings('ignore')

There are a lot of TTC code definitions (155, only 71 are in the dataset), and some of them are similar and can be grouped. We have manually assigned groups to TTC codes to make them larger

In [ ]:
ttc_groups = [1, 1, 2, 1, 1, 1, 1, 100, 101, 102, 4, 3, 3, 3, 3, 3, 4, 4, 4, 4, 5, 11, 11, 11, 103, 19, 19, 19, 19, 19, 19, 12, 12, 12, 12, 12, 104, 19, 19, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 105, 3, 57, 19, 19, 7, 10, 10, 10, 10, 106, 10, 107, 10, 10, 10, 10, 10, 10, 10, 10, 108, 10, 10, 109, 10, 10, 10, 10, 10, 10, 10, 2, 2, 2, 2, 110, 10, 10, 10, 10, 10, 10, 10, 111, 19, 17, 17, 17, 17, 17, 17, 13, 18, 18, 18, 18, 18, 18, 18, 18, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 13, 13, 9, 9, 9, 9, 9, 9, 112, 15, 15, 15, 15, 14, 22, 20, 19, 19, 9, 19, 16, 6, 113, 8, 21, 10, 10, 10, 10]
ttc['group'] = ttc_groups

In [ ]:
dfTrx = pd.merge(dfTrx, ttc, on='ttc', how="right")

Converting tm to pandas datetime type for convenience

In [ ]:
dfTrx['tm'] = pd.to_datetime(dfTrx['tm'], format="%H:%M:%S")

New columns:
- Weekday (dt modulo 7)
- Hour (0-23)
- Night (0/1, 00:00-05:00)

In [ ]:
dfTrx['weekday'] = dfTrx['ttc'] % 7
dfTrx['hour'] = dfTrx['tm'].dt.hour
dfTrx['night'] = ~dfTrx['hour'].between(5, 23).astype(int)

Separating transactions by
- Incoming transactions (amt above zero)
- Outgoing transactions (amt below zero)

In [ ]:
dfTrx.loc[dfTrx['amt'] > 0, 'income'] = dfTrx['amt']
dfTrx.loc[dfTrx['amt'] <= 0, 'spent'] = dfTrx['amt']

Creating new feature: unique TID rate (will further be applied to one model of two only). The means of TIDs

Formula for one user: $rate = \frac{unique}{total}$

In [ ]:
unique_tid = dfTrx[['tid', 'cid']].groupby('cid').nunique()
all_tid = dfTrx[['tid', 'cid']].groupby('cid').count()
unique_tid['tid_rate'] = unique_tid['tid'] / all_tid['tid']
unique_tid = unique_tid[['tid_rate']]

Calculating measures of central tendency for amount (income/spent):
- STD
- Minimum
- Maximum
- Mean
- Median
- Sum
- Count

In [ ]:
std_df = dfTrx.groupby(['cid']).std()
std_df.rename(columns={'income': 'std_income', 'spent': 'std_spent'}, inplace=True)

min_df = dfTrx.groupby(['cid']).min()
min_df.rename(columns={'income': 'min_income', 'spent': 'max_spent'}, inplace=True)

max_df = dfTrx.groupby(['cid']).max()
max_df.rename(columns={'income': 'max_income', 'spent': 'min_spent'}, inplace=True)

mean_df = dfTrx.groupby(['cid']).mean()
mean_df.rename(columns={'income': 'mean_income', 'spent': 'mean_spent'}, inplace=True)

median_df = dfTrx.groupby(['cid']).median()
median_df.rename(columns={'income': 'median_income', 'spent': 'median_spent'}, inplace=True)

sum_df = dfTrx.groupby(['cid']).sum()
sum_df.rename(columns={'income': 'sum_income', 'spent': 'sum_spent'}, inplace=True)

count_df = dfTrx.groupby(['cid']).count()
count_df.rename(columns={'income': 'count_income', 'spent': 'count_spent'}, inplace=True)

Creating new features for users with:
- Transaction count by MCC
- Transaction count by weekday
- Transaction count by hour
- Transaction count by night flag
- Transaction count by TTC group
- Transaction count by working hours flag

In [ ]:
def one_hot(x):
    features = []
    features.append(pd.Series(x['mcc'].value_counts().add_prefix('mcc_')))
    features.append(pd.Series(x['weekday'].value_counts().add_prefix('day_')))
    features.append(pd.Series(x['hour'].value_counts().add_prefix('h_')))
    features.append(pd.Series(x['night'].value_counts().add_prefix('night_')))
    features.append(pd.Series(x['group'].value_counts().add_prefix('new_ttc_')))

    return pd.concat(features)

data = dfTrx.groupby(dfTrx['cid']).apply(one_hot).unstack(-1)

Appending variance values to the final train dataset

In [ ]:
min_df = min_df[['min_income', 'max_spent']]
max_df = max_df[['max_income', 'min_spent']]
mean_df = mean_df[['mean_income', 'mean_spent']]
std_df = std_df[['std_income', 'std_spent']]
median_df = median_df[['median_income', 'median_spent']]
sum_df = sum_df[['sum_income', 'sum_spent']]
count_df = count_df[['count_income', 'count_spent']]

data = data.join(min_df, on='cid')
data = data.join(max_df, on='cid')
data = data.join(mean_df, on='cid')
data = data.join(std_df, on='cid')
data = data.join(median_df, on='cid')
data = data.join(sum_df, on='cid')
data = data.join(count_df, on='cid')
data = data.join(unique_tid, on='cid')

Replacing some remaining NaN features with zeros to further apply MinMaxScaler

In [ ]:
data.fillna(0, inplace=True)
data.head()

,day_0,day_1,day_2,day_3,day_4,day_5,day_6,h_0.0,h_1.0,h_10.0,...,mean_spent,std_income,std_spent,median_income,median_spent,sum_income,sum_spent,count_income,count_spent,tid_rate
cid,,,,,,,,,,,,,,,,,,,,,
0.0,0.0,634.0,73.0,16.0,178.0,103.0,1.0,178.0,6.0,26.0,...,-17222.883555,346823.277740,49156.154826,126887.5,-6123.0,4622047.0,-16861203.0,26,979,0.323636
1.0,0.0,17.0,67.0,12.0,53.0,60.0,0.0,7.0,0.0,19.0,...,-37538.816754,150208.983967,70819.531155,94620.5,-13675.0,2597151.0,-7169914.0,18,191,0.477124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3.0,0.0,2.0,0.0,8.0,17.0,50.0,1.0,18.0,0.0,12.0,...,-395424.457143,54271.935394,394880.084253,112303.0,-134749.0,1100510.0,-27679712.0,8,70,0.525000
4.0,0.0,219.0,13.0,0.0,38.0,17.0,0.0,0.0,0.0,32.0,...,-11620.638596,7942.223366,20777.662966,39302.0,-6692.0,78604.0,-3311882.0,2,285,0.217391


# Train/valid/test split

Binding target values to train dataset and extracting target values

In [ ]:
data_train = pd.merge(data, tY, on='cid', how="right")
data_test = pd.merge(data, vY, on='cid', how="right")

target = data_train['gender']
data_train.drop(columns='gender', inplace=True)
data_test.drop(columns='gender', inplace=True)

Splitting dataset to train and valid

In [ ]:
X = data_train.copy()
y = target
test_df = data_test.copy()

scaler = MinMaxScaler()
X = scaler.fit_transform(X)
test_df = scaler.fit_transform(test_df)

# Model learning

First, we fit XGBClassifier model on the current dataset. The hyperparameters were optimised with GridSearchCV and Optuna

In [ ]:
params_xgb = {'learning_rate':0.05,
 'n_estimators':1100,
 'max_depth':2,
 'min_child_weight':1,
 'gamma':2,
 'subsample':0.755,
 'colsample_bytree':0.8,
 'objective': 'binary:logistic',
 'nthread':4,
 'scale_pos_weight':1,
 'seed':27,
 'tree_method':'gpu_hist',
 'predictor': 'gpu_predictor'
}
model_xgb = xgb.XGBClassifier(**params_xgb)
model_xgb.fit(X, y)
y_pred = model_xgb.predict_proba(test_df)
res = [x[1] for x in y_pred]


submission_xgb = pd.DataFrame(index=data_test.index)
submission_xgb['probability'] = res
submission_xgb = submission_xgb.reset_index()

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=2, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=1100, n_jobs=None, nthread=4, num_parallel_tree=None,
              predictor='gpu_predictor', ...)

Next, we drop unique_tid column. This seems like a dumb decision, but we have achieved this with trial and error. After that, we have to apply MinMaxScaler again

In [ ]:
data = data.drop('tid_rate', axis=1)
data_train = pd.merge(data, tY, on='cid', how="right")
data_test = pd.merge(data, vY, on='cid', how="right")

target = data_train['gender']
data_train.drop(columns='gender', inplace=True)
data_test.drop(columns='gender', inplace=True)

X = data_train.copy()
y = target
test_df = data_test.copy()

scaler = MinMaxScaler()
X = scaler.fit_transform(X)
test_df = scaler.fit_transform(test_df)


Finally, we fit LGBMClassifier model on the new dataset (without TID rate). The hyperparameters were optimised with GridSearchCV. This model tends to give better results without this feature

In [ ]:
params_lgbm = {
    'metric': 'binary_logloss',
    'objective':'binary',
    'learning_rate': 0.05,
    'max_depth': 2,
    'n_estimators': 1000
}
model_lgbm = lightgbm.LGBMClassifier(**params_lgbm)
model_lgbm.fit(X, y)

y_pred = model_lgbm.predict_proba(test_df)
res = [x[1] for x in y_pred]


submission_lgbm = pd.DataFrame(index=data_test.index)
submission_lgbm['probability'] = res
submission_lgbm = submission_lgbm.reset_index()

LGBMClassifier(learning_rate=0.05, max_depth=2, metric='binary_logloss',
               n_estimators=1000, objective='binary')

To increase ROC AUC score on the private dataset, we blend two submissions together, applying weighted mean. XGBoost model give a slightly better score

In [ ]:
XGB_W = 0.55
LGBM_W = 1 - XGB_W
def blending(sub1, sub2, w1, w2):
    res = sub1.copy()
    res['probability'] = w1*sub1['probability'] + w2*sub2['probability']
    res['cid'] = res['cid'].astype(int)
    return res
blending(submission_xgb, submission_lgbm, XGB_W, LGBM_W).to_csv('submission.csv', index=False)

<font size=5>⌛</font> <strong><font color=green size=5>Do not exceed competition's runtime limit! Do not write code outside TGP</font></strong>
<hr color=green size=40>

In [ ]:
tmr.ShowTime()    # measure Colab's runtime. Do not remove. Keep as the last cell in your notebook.

Runtime is 74 sec
